# 9장 텐서플로 시작하기

텐서플로는 오픈소스 라이버리로 딥러닝에 매우 유용한 툴이다. 
딥러닝은 keras와 pytorch 등 다른 라이브러리가 존재하지만 가장 대중적이다.
Tensorflow의 대표적인 자료형 tf.constant tf.variable tf.placeholder가 존재
constant : 텐서플로우 변하지 않는 상수 생수
varialble : 값이 바뀔 수도 있는 변수 생성
placeholder : 일정 값을 받을 수 있게 만들어주는 그릇 생성

In [1]:
import tensorflow as tf
tf.__version__

'1.14.0'

In [2]:
x = tf.Variable(3, name = "x")
y = tf.Variable(4, name = 'y')
f = x*x*y + y+ 2 #실제로 어떤 연산도 되지 않음. sess으로 cpu혹은 gpu에 올려주어야함

In [3]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)

In [4]:
print(result)

42


In [5]:
sess.close()

In [6]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval() # tf.get_default_session().run(f)를 하는것과 동일 # with블록 끝나면 session 자동 종료

In [7]:
init = tf.global_variables_initializer() # 변수 초기화 일일이 하지 않고 계산 그래프가 실행될때 모든 변수 초기화할 노드 생성
with tf.Session() as sess:
    init.run() # 실제 모든 변수 초기화
    result = f.eval() # with 없으면 수동 종료 필요

In [8]:
print(result)

42


In [9]:
# 9.3 계산 그래프 관리
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [10]:
# 독립된 연산을 하려면
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [11]:
x2.graph is graph

True

In [12]:
x2.graph is tf.get_default_graph()

False

In [13]:
# 9.4노드 값의 생애주기
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval()) # 2번 저장돼

10
15


In [14]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


In [15]:
# 9.5 텐서플로를 이용한 선형 회귀 - 정규방정식 이용
import numpy as np
from sklearn.datasets import fetch_california_housing

In [16]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

In [17]:
X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name="y")
XT = tf.transpose(X)

In [18]:
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)),XT), y) 

with tf.Session() as sess:
    theta_value = theta.eval()

print(theta_value)

[[-3.74659576e+01]
 [ 4.35208052e-01]
 [ 9.34183039e-03]
 [-1.05619654e-01]
 [ 6.38267040e-01]
 [-4.28281601e-06]
 [-3.77140474e-03]
 [-4.26884502e-01]
 [-4.40567464e-01]]


9.6 경사하강법 구현

In [19]:
from sklearn.preprocessing import StandardScaler # 경사하강법을 위해서는 정규화가 필요

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [20]:
#reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
W = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='W') # 기존 칼럼 + 상수

y_pred = tf.matmul(X, W, name='predictions')  # y = wx + b
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(W, W - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("epoch: {:03d}, MSE = {:.6f}".format(epoch, mse.eval()))
        sess.run(training_op)
        
    best_W = W.eval()  # sess.run(W)

print(best_W)

epoch: 000, MSE = 2.754426
epoch: 100, MSE = 0.632222
epoch: 200, MSE = 0.572780
epoch: 300, MSE = 0.558501
epoch: 400, MSE = 0.549070
epoch: 500, MSE = 0.542288
epoch: 600, MSE = 0.537379
epoch: 700, MSE = 0.533822
epoch: 800, MSE = 0.531242
epoch: 900, MSE = 0.529370
[[ 2.0685523e+00]
 [ 7.7407807e-01]
 [ 1.3119239e-01]
 [-1.1784514e-01]
 [ 1.6477820e-01]
 [ 7.4408232e-04]
 [-3.9194513e-02]
 [-8.6135650e-01]
 [-8.2347965e-01]]


In [21]:
# 자동미분
def my_func(a, b):
    z = 0
    for i in range(100):
        z = a*np.cos(z+ i) + z * np.sin(b-i)
        return z

In [22]:
gradients = tf.gradients(mse, [theta])[0]
gradients

In [23]:
# 옵티마이저 사용
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)

9.7 훈련 알고리즘에 데이터 주입 

In [24]:
# 미니배치 사용 #placeholder 사용 - 실제로 아무 계산도 하지 않는 특수 노드 훈련데이터 전달하기 위해 사용
A = tf.placeholder(tf.float32, shape = (None, 3)) 
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict = {A: [[1,2,3]]})
    B_val_2 = B.eval(feed_dict = {A :[[4,5,6],[7,8,9]]})
    
print(B_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


In [29]:

# placeholder setting
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")  # (None, 9)
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

W = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='W')

y_pred = tf.matmul(X, W, name='predictions')  # y = wx + b
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

# optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)


In [30]:
################
# hyper-params #
################
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))  # (20640/100) 올림
# mini-batch random sampling
def fetch_batch(epoch, batch_index, batch_size):
    """ epoch당 batch_size만큼 Random sampling 하는 메소드
    
    Args:
        - epoch: training epoch
        - batch_index: batch steps
        - batch_size: batch size
    
    Returns:
        - X_batch, y_batch: batch size만큼의 X, y 데이터셋
    """    
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch


In [32]:

# Training
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
    best_W = W.eval()
    
print(best_W)

[[ 2.070016  ]
 [ 0.820456  ]
 [ 0.1173173 ]
 [-0.22739047]
 [ 0.31134018]
 [ 0.00353193]
 [-0.01126995]
 [-0.9164394 ]
 [-0.8795008 ]]


 9.8 모델 저장과 복원

In [28]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
W = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='W')

y_pred = tf.matmul(X, W, name='predictions')  # y = wx + b
error = y_pred - y

# loss function
mse = tf.reduce_mean(tf.square(error), name='mse')
# optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver(max_to_keep=3)
tf.add_to_collection('weight', W)

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("epoch: {:03d}, MSE = {:.6f}".format(epoch, mse.eval()))
            save_path = saver.save(sess, './model/my_model.ckpt')
        sess.run(training_op)
        
    best_W = W.eval()  # sess.run(W)
    save_path = saver.save(sess, "./model/my_model_final.ckpt")

print(best_W)

epoch: 000, MSE = 2.754426
epoch: 100, MSE = 0.632222
epoch: 200, MSE = 0.572780
epoch: 300, MSE = 0.558501
epoch: 400, MSE = 0.549070
epoch: 500, MSE = 0.542288
epoch: 600, MSE = 0.537379
epoch: 700, MSE = 0.533822
epoch: 800, MSE = 0.531242
epoch: 900, MSE = 0.529370
[[ 2.0685525e+00]
 [ 7.7407807e-01]
 [ 1.3119237e-01]
 [-1.1784511e-01]
 [ 1.6477817e-01]
 [ 7.4407971e-04]
 [-3.9194521e-02]
 [-8.6135668e-01]
 [-8.2347977e-01]]


In [35]:
# 9.8 모델 저장과 복원 : 모델을 훈련시키면 다시 쓸수 있도록 파라미터 저장
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
W = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='W')

y_pred = tf.matmul(X, W, name='predictions')  # y = wx + b
error = y_pred - y

# loss function
mse = tf.reduce_mean(tf.square(error), name='mse')
# optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver(max_to_keep=3)
tf.add_to_collection('weight', W)

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("epoch: {:03d}, MSE = {:.6f}".format(epoch, mse.eval()))
            save_path = saver.save(sess, './model/my_model.ckpt')
        sess.run(training_op)
        
    best_W = W.eval()  # sess.run(W)
    save_path = saver.save(sess, "./model/my_model_final.ckpt")

print(best_W)

epoch: 000, MSE = 2.754426
epoch: 100, MSE = 0.632222
epoch: 200, MSE = 0.572780
epoch: 300, MSE = 0.558501
epoch: 400, MSE = 0.549070
epoch: 500, MSE = 0.542288
epoch: 600, MSE = 0.537379
epoch: 700, MSE = 0.533822
epoch: 800, MSE = 0.531242
epoch: 900, MSE = 0.529370
[[ 2.0685525e+00]
 [ 7.7407807e-01]
 [ 1.3119237e-01]
 [-1.1784511e-01]
 [ 1.6477817e-01]
 [ 7.4407971e-04]
 [-3.9194521e-02]
 [-8.6135668e-01]
 [-8.2347977e-01]]


In [36]:
with tf.Session() as sess:
    saver.restore(sess, './model/my_model_final.ckpt')
    best_W_restored = tf.get_collection('weight')[0]
    best_W_restored = best_W_restored.eval()
    
# compare element-wise equal or not
print(np.allclose(best_W, best_W_restored))  # True

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model/my_model_final.ckpt
False


 9.9 텐서보드로 그래프와 학습 곡선 시각화 하기

In [ ]:
# 실행이 안됨 - ValueError: Duplicate plugins for name projector
# - tensorflow랑 tensorboard version이 안맞아서. 
#pip uninstall tensorboard
#pip install 버전맞게 다운

In [41]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [42]:
# placeholder setting
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")  # (None, 9)
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

W = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='W')

y_pred = tf.matmul(X, W, name='predictions')  # y = wx + b
error = y_pred - y

# loss function
mse = tf.reduce_mean(tf.square(error), name='mse')
# optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [43]:
# tensorboard에서 요약 정보를 보기위한 설정
mse_summary = tf.summary.scalar('MSE', mse)

In [44]:

################
# hyper-params #
################
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))  # (20640/100) 올림

In [45]:
with tf.Session() as sess:
    sess.run(init)
    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, 
                                                          y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            
            sess.run(training_op, feed_dict={X: X_batch,
                                             y: y_batch})
            
    best_W = W.eval()
    
print(best_W)

[[ 2.070016  ]
 [ 0.820456  ]
 [ 0.1173173 ]
 [-0.22739047]
 [ 0.31134018]
 [ 0.00353193]
 [-0.01126995]
 [-0.9164394 ]
 [-0.8795008 ]]
